Команды для запуска hadoop

In [ ]:
cd
hadoop-3.2.3/bin/hdfs namenode -format
export PDSH_RCMD_TYPE=ssh
start-all.sh
hadoop fs -mkdir /user/
hadoop fs -mkdir /user/abobus
hadoop fs -put ~/Downloads/u.data /user/abobus
hadoop fs -put ~/Downloads/u.item /user/abobus

In [1]:
import pyspark as ps
import json

from pyspark import *

Переменные, которые будем использовать далее

In [2]:
mark_path = 'hdfs://localhost:9000/user/abobus/u.data'
movies_path = 'hdfs://localhost:9000/user/abobus/u.item'
my_num = 212 + 25

In [3]:
spark = ps.sql.SparkSession.builder.appName('xxx').getOrCreate()

22/12/12 03:45:11 WARN Utils: Your hostname, BigData resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/12 03:45:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 03:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Читаем файлы

In [4]:
movies_ds = spark.read.csv(movies_path, sep = '|', encoding = 'cp1252', header = None)  # Читаем файл, задаем разделитель, устанавливаем кодировку, отбрасываем заголовки
movies_ds = movies_ds.drop(*movies_ds.columns[2 : len(movies_ds.columns)])              # Убираем лишние столбцы, распаковывая список названий столбцов со второго до последнего
movies_ds = movies_ds.withColumnRenamed('_c0', 'mID')                                   # Переименовываем столбец
movies_ds = movies_ds.withColumnRenamed('_c1', 'Movie_name')                            # Переименовываем столбец

In [5]:
marks_ds = spark.read.csv(mark_path, sep = '\t')                                        # Читаем файл, задаем разделитель

marks_ds = marks_ds.withColumnRenamed('_c0', 'uID')                                     # Переименовываем столбец
marks_ds = marks_ds.withColumnRenamed('_c1', 'mID')                                     # Переименовываем столбец
marks_ds = marks_ds.withColumnRenamed('_c2', 'mark')                                    # Переименовываем столбец
marks_ds = marks_ds.withColumnRenamed('_c3', 'timestamp')                               # Переименовываем столбец

In [6]:
movies_ds.createOrReplaceTempView('Movies_db')                                          # Создаем представление таблицы с именем Movies_db
marks_ds.createOrReplaceTempView('Marks_db')                                            # Создаем представление таблицы с именем Marks_db

В подзапросе выбираем id фильма, пользователя и оценку, которую поставил данный пользователь на этот фильм
Было использовано объединение, учитывающее только пересекающиеся значения. Объединены таблица фильмов и оценок по айди фильма, где айди фильма - номер, выданный по заданию

In [7]:
spark.sql(f'''
with movie_mark (
        select Movies_db.mID, uID, mark
        from 
            Movies_db inner join Marks_db on Movies_db.mID = Marks_db.mID
        where Movies_db.mID = {my_num}
    )

select mark, count(mark) as quantity
from movie_mark
group by mark
order by mark
''').show()

+----+--------+
|mark|quantity|
+----+--------+
|   1|       6|
|   2|      36|
|   3|      96|
|   4|     171|
|   5|      75|
+----+--------+



В подзапросе выбираем айди фильма, пользователя и оценку фильма пользователем из объединенных таблиц фильмов и оцеок, где айли фильма совпадает с выданным по заданию номером.
Выбираем оценки, количество оценок, группируем по оценке

In [8]:
marks_237 = spark.sql(f'''
    with movie_mark (
            select Movies_db.mID, uID, mark
            from 
                Movies_db inner join Marks_db 
                    on Movies_db.mID = Marks_db.mID
                where Movies_db.mID = {my_num}
        )

    select mark, count(mark) as quantity
    from movie_mark
        group by mark
        order by mark
''').collect()

В подзапросе выбираем название фильма, айди пользователя и его оценку фильму из объединения таблиц фильмов и оценок по совпадению айди фильма.
Выбираем оценки, количество оценок из подзапроса, группируем по оценке

In [9]:
marks_all = spark.sql(f'''

with movie_mark (
        select Movies_db.Movie_name, uID, mark
        from 
            Movies_db inner join Marks_db 
                on Movies_db.mID = Marks_db.mID
    )
    select mark, count(mark) as quantity
    from movie_mark
        group by mark
        order by mark
    
''').collect()

Переписываем результаты запросов в списки

In [10]:
marks_237 = [res[1] for res in marks_237]
marks_all = [res[1] for res in marks_all]

Открываем json файл на запись, сохраняем в него словарь, где есть два ключа - marks_237 для фильма, чей айди был дан по заданию и marks_all для сбора сведений об оценках всех фильмов

In [15]:
with open('db_lab_3.json', 'w') as json_file:
    json.dump(json.dumps({ 'rates_filmID_237' : marks_237, 'rates_films' : marks_all }), json_file)

Как выглядят записываемые данные

In [17]:
json.dumps({ 'rates_filmID_237' : marks_237, 'rates_films' : marks_all })

'{"rates_filmID_237": [6, 36, 96, 171, 75], "rates_films": [6110, 11370, 27145, 34174, 21201]}'